# Iris Dataset Example - Decision Tree

In [18]:
import sys

!{sys.executable} -m pip install skl2onnx==1.17.0
!{sys.executable} -m pip install onnx==1.16.1
!{sys.executable} -m pip install onnxruntime==1.18.1


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [21]:
from skl2onnx import to_onnx
from skl2onnx.common.data_types import FloatTensorType
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import onnxruntime as rt
import numpy as np

# create a pipeline object
pipe = make_pipeline(
    StandardScaler(),
    DecisionTreeClassifier()
)

# load the iris dataset and split it into train and test sets
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# fit the whole pipeline
pipe.fit(X_train, y_train)

# we can now use it like any other estimator
print(accuracy_score(pipe.predict(X_test), y_test))

onxx_pipeline = to_onnx(pipe, initial_types=[('sepalLength', FloatTensorType([None, 1])),
                                             ('sepalWidth', FloatTensorType([None, 1])),
                                             ('petalLength', FloatTensorType([None, 1])),
                                             ('petalWidth', FloatTensorType([None, 1]))])

with open("iris_dt.onnx", "wb") as f:
    f.write(onxx_pipeline.SerializeToString())

0.9736842105263158


In [22]:
sess = rt.InferenceSession("iris_dt.onnx")
inputs = {'sepalLength': [[5.0]], 'sepalWidth': [[3.5]], 'petalLength': [[4.0]], 'petalWidth': [[5.0]]}

pred_onx = sess.run(None, inputs)

In [23]:
print("prediction", pred_onx[0])
print("probabilites", pred_onx[1])

prediction [1]
probabilites [{0: 0.0, 1: 1.0, 2: 0.0}]
